In [50]:
import pandas as pd
import numpy as np

### Carga del dataset

In [51]:
df = pd.read_csv("/home/bigdata/Integrated-EDA-/data/Gaming_Mental_Health.csv")

### Limpieza de nombres de juegos que será la key para juntarlo con la Api en porcentaje

In [52]:
null_counts = (df.isnull().sum() / len(df)) * 100
print(null_counts)

record_id                            0.0
age                                  0.0
gender                               0.0
daily_gaming_hours                   0.0
game_genre                           0.0
primary_game                         0.0
gaming_platform                      0.0
sleep_hours                          0.0
sleep_quality                        0.0
sleep_disruption_frequency           0.0
academic_work_performance            0.0
grades_gpa                          24.6
work_productivity_score             32.6
mood_state                           0.0
mood_swing_frequency                 0.0
withdrawal_symptoms                  0.0
loss_of_other_interests              0.0
continued_despite_problems           0.0
eye_strain                           0.0
back_neck_pain                       0.0
weight_change_kg                     0.0
exercise_hours_weekly                0.0
social_isolation_score               0.0
face_to_face_social_hours_weekly     0.0
monthly_game_spe

Tratar estas columnas "grades_gpa" "work_productivity_score" lo mas seguro será eliminarlos.

In [53]:
print(df.nunique())

record_id                           1000
age                                   23
gender                                 3
daily_gaming_hours                   132
game_genre                             7
primary_game                          24
gaming_platform                        4
sleep_hours                           61
sleep_quality                          5
sleep_disruption_frequency             5
academic_work_performance              6
grades_gpa                           275
work_productivity_score               10
mood_state                             9
mood_swing_frequency                   5
withdrawal_symptoms                    2
loss_of_other_interests                2
continued_despite_problems             2
eye_strain                             2
back_neck_pain                         2
weight_change_kg                      70
exercise_hours_weekly                 90
social_isolation_score                10
face_to_face_social_hours_weekly     158
monthly_game_spe

Con esto se los valores unicos y tener una ligera idea de las columnas y podemos buscarlos cada columna

No tenemos ninguna fila duplicada.

In [63]:
print(df['gaming_platform'].value_counts())

gaming_platform
Mobile            262
Multi-platform    260
PC                241
Console           237
Name: count, dtype: int64


In [ ]:
df['gaming_platform']= df[df[""]]

In [55]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 27 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   record_id                         1000 non-null   str    
 1   age                               1000 non-null   int64  
 2   gender                            1000 non-null   str    
 3   daily_gaming_hours                1000 non-null   float64
 4   game_genre                        1000 non-null   str    
 5   primary_game                      1000 non-null   str    
 6   gaming_platform                   1000 non-null   str    
 7   sleep_hours                       1000 non-null   float64
 8   sleep_quality                     1000 non-null   str    
 9   sleep_disruption_frequency        1000 non-null   str    
 10  academic_work_performance         1000 non-null   str    
 11  grades_gpa                        754 non-null    float64
 12  work_productivity_

In [56]:
# En lugar de inventar datos, marcamos los que faltan o imputamos la mediana
df['grades_gpa'] = df['grades_gpa'].fillna(df['grades_gpa'].median())
df['work_productivity_score'] = df['work_productivity_score'].fillna(0) # Asumimos 0 si no hay respuesta


In [60]:
df['primary_game'] = df['primary_game'].str.upper()

## Conexion con SteamSpy

In [61]:
import requests
import time
import pandas as pd

def get_steamspy_data(game_name):
    # 1. Intentamos búsqueda por término
    search_url = f"https://steamspy.com/api.php?request=search&term={game_name}"
    
    try:
        response = requests.get(search_url).json()
        
        # SteamSpy devuelve un dict con AppIDs como llaves si encuentra algo
        if response and isinstance(response, dict) and len(response) > 0:
            # Tomamos el primer AppID de los resultados
            first_appid = list(response.keys())[0]
            
            # 2. Pedimos los detalles de ese ID
            detail_url = f"https://steamspy.com/api.php?request=appdetails&appid={first_appid}"
            details = requests.get(detail_url).json()
            
            print(f"✅ Encontrado: {game_name} (AppID: {first_appid})")
            return {
                'primary_game_clean': game_name,
                'average_forever': details.get('average_forever', 0),
                'owners': details.get('owners', '0').replace(',', ''), # Limpiamos comas
                'price': details.get('price', '0')
            }
        else:
            print(f"❌ No encontrado en Steam: {game_name}")
            return None
    except Exception as e:
        print(f"⚠️ Error con {game_name}: {e}")
        return None

# --- EJECUCIÓN ---
# Probamos con una muestra que sepamos que está en Steam para verificar
top_games = df['primary_game'].value_counts().index[:15]
steam_data = []

print("Iniciando consulta a SteamSpy...")
for game in top_games:
    data = get_steamspy_data(game)
    if data:
        steam_data.append(data)
    time.sleep(1.1) # Un poco más de margen por seguridad

df_steam = pd.DataFrame(steam_data)

if df_steam.empty:
    print("\n🚨 ALERTA: df_steam sigue vacío. Es probable que los juegos del top no estén en Steam.")
else:
    print(f"\n✨ Éxito: Se han obtenido datos de {len(df_steam)} juegos.")

Iniciando consulta a SteamSpy...
⚠️ Error con DOTA 2: Expecting value: line 1 column 1 (char 0)
⚠️ Error con ELDEN RING: Expecting value: line 1 column 1 (char 0)
⚠️ Error con STARCRAFT II: Expecting value: line 1 column 1 (char 0)
⚠️ Error con WORLD OF WARCRAFT: Expecting value: line 1 column 1 (char 0)
⚠️ Error con CIVILIZATION VI: Expecting value: line 1 column 1 (char 0)
⚠️ Error con MOBILE LEGENDS: Expecting value: line 1 column 1 (char 0)
⚠️ Error con LEAGUE OF LEGENDS: Expecting value: line 1 column 1 (char 0)
⚠️ Error con FINAL FANTASY XIV: Expecting value: line 1 column 1 (char 0)
⚠️ Error con CYBERPUNK 2077: Expecting value: line 1 column 1 (char 0)
⚠️ Error con ELDER SCROLLS ONLINE: Expecting value: line 1 column 1 (char 0)
⚠️ Error con SKYRIM: Expecting value: line 1 column 1 (char 0)
⚠️ Error con FORTNITE: Expecting value: line 1 column 1 (char 0)
⚠️ Error con CLASH OF CLANS: Expecting value: line 1 column 1 (char 0)
⚠️ Error con WARZONE: Expecting value: line 1 column 1 (

In [58]:
df_steam.head()

""


In [59]:
# Unimos por la columna 'primary_game_clean'
df_final = pd.merge(df, df_steam, on='primary_game_clean', how='left')

df_final.to_csv('/home/bigdata/Integrated-EDA-/data/gaming.csv', index=False)

KeyError: 'primary_game_clean'